In [1]:
from openpyxl import load_workbook
from collections import defaultdict
import json
from helpers.histories import bulk_update_with_history
from rest_framework_json_api.renderers import JSONRenderer
from apps.reportings.serializers import ReportingSerializer
from apps.reportings.views import ReportingView
from json_logic import jsonLogic

In [2]:
!cat .env

STAGE=LOCAL


In [3]:
company_name = "Eixo SP"
companies = Company.objects.filter(name=company_name)
company = companies.first()

In [4]:
prefixo_folder = "fixtures/roads"
excel_file = "TrechoRodovias_Seg_Conserva.xlsx"

In [5]:
wb = load_workbook('{}/{}'.format(prefixo_folder, excel_file))
sheetname = wb.sheetnames[0]
ws = wb[sheetname]

In [6]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

In [7]:
values

[{'Rodovia': 'SP-191',
  'Tipo': 'SP',
  'Código': '191 - Trecho I',
  'km inicial': 74.72,
  'km inicial arred': '=ROUNDDOWN(D2,0)',
  'km final': 98.49,
  'km final arred': '=ROUNDUP(F2,0)',
  'Extenção (km)': 23.77,
  'Tipo de pista': 'Simples',
  'Regional': '1.São Pedro',
  'Sistema': 'Existente'},
 {'Rodovia': 'SP-191',
  'Tipo': 'SP',
  'Código': '191 - Trecho II',
  'km inicial': 99,
  'km inicial arred': '=ROUNDDOWN(D3,0)',
  'km final': 115.61,
  'km final arred': '=ROUNDUP(F3,0)',
  'Extenção (km)': 16.61,
  'Tipo de pista': 'Simples',
  'Regional': '1.São Pedro',
  'Sistema': 'Existente'},
 {'Rodovia': 'SPI-083/191',
  'Tipo': 'SPI',
  'Código': '083/191',
  'km inicial': 0,
  'km inicial arred': '=ROUNDDOWN(D4,0)',
  'km final': 10,
  'km final arred': '=ROUNDUP(F4,0)',
  'Extenção (km)': 10,
  'Tipo de pista': 'Simples',
  'Regional': '1.São Pedro',
  'Sistema': 'Existente'},
 {'Rodovia': 'SP-197',
  'Tipo': 'SP',
  'Código': 197,
  'km inicial': 0,
  'km inicial arred': 

In [8]:
road_types = defaultdict(list)
road_systems = defaultdict(list)

In [9]:
lane_types = company.metadata["csp"].get("lane_types", {})
lane_systems = company.metadata["csp"].get("lane_systems", {})

In [10]:
for value in values:
    road_types[value['Rodovia']].append((value['km inicial'], value['km final'], lane_types[value['Tipo de pista']]))
    road_systems[value['Rodovia']].append((value['km inicial'], value['km final'], lane_systems[value['Sistema']]))

In [11]:
road_types

defaultdict(list,
            {'SP-191': [(74.72, 98.49, '1'), (99, 115.61, '1')],
             'SPI-083/191': [(0, 10, '1')],
             'SP-197': [(0, 20.02, '1')],
             'SPA-005/197': [(0, 4.75, '1')],
             'SP-304': [(256.76, 293.91, '1'),
              (198.14, 256.76, '1'),
              (168.512, 188, '2'),
              (188, 198.14, '1')],
             'SPA-195/304': [(0, 3, '1')],
             'SPA-198/304': [(0, 0.65, '2')],
             'SPA-201/304': [(0, 1.98, '2')],
             'SPA-223/304': [(0, 2.1, '2')],
             'SPA-227/304': [(0, 1, '1')],
             'SP-308': [(173.622, 178, '2'), (178, 194.2, '1')],
             'SPI-194/308': [(0, 3.1, '1')],
             'SP-225': [(91.43, 235.04, '2')],
             'SPA-130/225': [(0, 1.08, '2')],
             'SPA-159/225': [(0, 10.7, '1')],
             'SPA-160/225': [(0, 1.9, '1')],
             'SPA-195/225': [(0, 7.77, '1')],
             'SPA-228/225': [(0.39, 4.8, '2')],
             'SP-261

In [12]:
road_systems

defaultdict(list,
            {'SP-191': [(74.72, 98.49, '1'), (99, 115.61, '1')],
             'SPI-083/191': [(0, 10, '1')],
             'SP-197': [(0, 20.02, '1')],
             'SPA-005/197': [(0, 4.75, '1')],
             'SP-304': [(256.76, 293.91, '1'),
              (198.14, 256.76, '1'),
              (168.512, 188, '1'),
              (188, 198.14, '1')],
             'SPA-195/304': [(0, 3, '1')],
             'SPA-198/304': [(0, 0.65, '1')],
             'SPA-201/304': [(0, 1.98, '1')],
             'SPA-223/304': [(0, 2.1, '1')],
             'SPA-227/304': [(0, 1, '1')],
             'SP-308': [(173.622, 178, '1'), (178, 194.2, '1')],
             'SPI-194/308': [(0, 3.1, '1')],
             'SP-225': [(91.43, 235.04, '2')],
             'SPA-130/225': [(0, 1.08, '2')],
             'SPA-159/225': [(0, 10.7, '2')],
             'SPA-160/225': [(0, 1.9, '2')],
             'SPA-195/225': [(0, 7.77, '2')],
             'SPA-228/225': [(0.39, 4.8, '2')],
             'SP-261

In [13]:
km_var = {"var": "data.km"}
km_var_2 = {"var": "km"}

In [14]:
road_types_logics = {}
road_systems_logics = {}

In [15]:
for road, kms in road_types.items():
    all_conditions = []
    conditions = [[{"or": [{"and": [{">=": [km_var, km[0]]}, {"<=": [km_var, km[1]]}]}, {"and": [{">=": [km_var_2, km[0]]}, {"<=": [km_var_2, km[1]]}]}]}, str(km[2])] for km in kms]
    for condition in conditions:
        all_conditions += condition
    all_conditions.append(None)
    road_types_logics[road] = {
        "if": all_conditions
    }
    
for road, kms in road_systems.items():
    all_conditions = []
    conditions = [[{"or": [{"and": [{">=": [km_var, km[0]]}, {"<=": [km_var, km[1]]}]}, {"and": [{">=": [km_var_2, km[0]]}, {"<=": [km_var_2, km[1]]}]}]}, str(km[2])] for km in kms]
    for condition in conditions:
        all_conditions += condition
    all_conditions.append(None)
    road_systems_logics[road] = {
        "if": all_conditions
    }

In [16]:
print(json.dumps(road_types_logics, indent=4))

{
    "SP-191": {
        "if": [
            {
                "or": [
                    {
                        "and": [
                            {
                                ">=": [
                                    {
                                        "var": "data.km"
                                    },
                                    74.72
                                ]
                            },
                            {
                                "<=": [
                                    {
                                        "var": "data.km"
                                    },
                                    98.49
                                ]
                            }
                        ]
                    },
                    {
                        "and": [
                            {
                                ">=": [
                                    {
                                        

In [17]:
print(json.dumps(road_systems_logics, indent=4))

{
    "SP-191": {
        "if": [
            {
                "or": [
                    {
                        "and": [
                            {
                                ">=": [
                                    {
                                        "var": "data.km"
                                    },
                                    74.72
                                ]
                            },
                            {
                                "<=": [
                                    {
                                        "var": "data.km"
                                    },
                                    98.49
                                ]
                            }
                        ]
                    },
                    {
                        "and": [
                            {
                                ">=": [
                                    {
                                        

In [20]:
all_roads = list(set(list(road_types_logics.keys()) + list(road_systems_logics.keys())))

In [22]:
for road in all_roads:
    type_logic = road_types_logics.get(road, {})
    system_logic = road_systems_logics.get(road, {})
    both_logic = {
        "type_logic": type_logic,
        "system_logic": system_logic
    }
    print(road, Road.objects.filter(company=company, name=road).update(lane_type_logic=both_logic))

SPA-578/294 1
SPI-083/191 3
SPA-517/284 1
SP-425 2
SPA-005/197 1
SPA-195/331 1
SPA-193/310 1
SPA-658/294 1
SPA-160/225 1
SPA-196/331 1
SPA-504/284 1
SPA-017/293 1
SP-191 2
SP-293 1
SPA-683/294 1
SP-331 1
SPA-431/425 1
SPA-228/225 1
SPA-592/294 1
SPA-201/304 1
SPA-605/294 1
SPA-511/294 1
SP-197 1
SPA-198/304 1
SPA-402/294 1
SP-310 1
SPA-130/225 1
SPA-227/304 1
SPA-498/284 1
SP-308 1
SP-304 3
SP-225 1
SPA-571/294 1
SPA-159/225 1
SPA-498/294 1
SPA-200/310 2
SPA-616/294 1
SPA-195/304 1
SPA-585/294 1
SPA-617/294 1
SPA-627/294 1
SP-261 1
SPA-195/225 1
SPA-360/294 1
SPA-437/425 1
SPA-006/293 1
SPA-475/284 1
SP-294 3
SP-284 2
SPA-223/304 1
SPI-194/308 1
SPA-570/294 2
